In [1]:
library(tidyverse)
library(RPostgreSQL)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "sql")
list.files(sql_dir)

[1] "basics.sql"              "cohort.sql"             
[3] "icd9.sql"                "lab_tests.sql"          
[5] "lab_unpivot.sql"         "merge_data.sql"         
[7] "population.sql"          "vital_signs.sql"        
[9] "vital_signs_unpivot.sql"

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
sql <- readr::read_file(file.path(sql_dir, "vital_signs.sql")) %>%
    make_view_sql("vital_signs", "materialized view")

In [6]:
dbSendQuery(con, sql)

<PostgreSQLResult>

In [7]:
sql <- "
select * from vital_signs limit 10;
"
dbGetQuery(con, sql)

icustay_id,charttime,label,valuenum
211552,2101-10-21 07:05:00,map,68
211552,2101-10-21 07:05:00,cvp,10
211552,2101-10-21 08:00:00,map,67
211552,2101-10-21 08:00:00,cvp,10
254478,2108-08-23 20:00:00,map,68
254478,2108-08-23 21:00:00,map,79
211552,2101-10-20 20:30:00,map,61
211552,2101-10-20 20:45:00,map,67
211552,2101-10-20 21:00:00,map,55
211552,2101-10-20 21:15:00,map,66


In [8]:
sql <- readr::read_file(file.path(sql_dir, "lab_tests.sql")) %>%
    make_view_sql("lab_tests", "materialized view")

In [9]:
dbSendQuery(con, sql)

<PostgreSQLResult>

In [10]:
sql <- "
select * from lab_tests limit 10;
"
dbGetQuery(con, sql)

hadm_id,charttime,label,valuenum,abnormal
145834,2101-10-20 19:26:00,bun,41.0,1
145834,2101-10-20 19:26:00,hemoglobin,7.8,1
145834,2101-10-20 19:26:00,platelet,190.0,0
145834,2101-10-20 19:26:00,wbc,11.3,1
145834,2101-10-20 19:59:00,bicarbonate,11.0,1
145834,2101-10-20 19:59:00,chloride,111.0,1
145834,2101-10-20 19:59:00,creatinine_kinase,82.0,0
145834,2101-10-20 19:59:00,creatinine,2.5,1
145834,2101-10-20 19:59:00,potassium,4.1,0
145834,2101-10-20 19:59:00,sodium,140.0,0


In [11]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE